# Databáze

V této lekci si ukážeme, jak se dá z pandas připojit k databázi. To nám umožní psát v rámci Pythonu SQL dotazy a jako výsledky dostávat dataframes. Ukážeme si pár příkladů a porovnáme rozdíly mezi zpracováním dat v databázi a v Pandas.

Nejdřív si ale ukážeme nějakou další práci s textovými řetězci, která se nám bude hodit při psaní SQL dotazů.

## 1. Víceřádkové stringy a další formátování

### Víceřádkové stringy

SQL dotazy se v Pythonu píšou jako obyčejné stringy. Jupyter to neumí, ale chytřejší programy poznají že je ve stringu SQL a zvýrazní nám jeho syntaxi.

Pro snazší čitelnost je, stejně jako v klasickém SQL, zvykem psát dotazy rozdělené do více řádků. Začneme tedy ukázkou, jak psát v Pythonu víceřádkové stringy.

Pokud ukončíme řádek uprostřed obyčejného stringu, chápe to Python jako oddělení dvou příkazů, a začne si stěžovat na neukončené uvozovky.

In [17]:
tohle_nepujde = "SELECT sloupec
FROM tabulka"

print(tohle_nepujde)

SyntaxError: EOL while scanning string literal (<ipython-input-17-18a9d138385f>, line 1)

Můžeme tam sice přidat znak `\n` jako nový řádek, ale ten funguje jen po vypsání na obrazovku nebo do souboru. Čitelnosti kódu to nijak nepomůže.

In [16]:
tohle_neni_citelne = "SELECT sloupec\nFROM tabulka"

print(tohle_neni_citelne)

SELECT sloupec
FROM tabulka


Python umožňuje začít (a ukončit) string třemi uvozovkami. Takový string pak chápe jako víceřádkový.

In [18]:
tohle_je_ok = """
SELECT sloupec
FROM tabulka
"""

print(tohle_je_ok)


SELECT sloupec
FROM tabulka



Takový string dokonce zachovává odsazení.

In [20]:
odsazeny_string = """
SELECT sloupec1,
       sloupec2,
FROM tabulka
WHERE sloupec3 IN (
    SELECT sloupec4
    FROM tabulka2
)
"""

print(odsazeny_string)


SELECT sloupec1,
       sloupec2,
FROM tabulka
WHERE sloupec3 IN (
    SELECT sloupec4
    FROM tabulka2
)



Občas si lidé myslí, že tři uvozovky znamenají víceřádkový komentář. To ale není pravda, Python žádné víceřádkové komentáře nemá.

In [21]:
"""
Zde není komentář.

Zde už vůbec ne.
"""

'\nZde není komentář.\n\nZde už vůbec ne.\n'

Jak vidíme z výstupu Jupyteru, Python vytvořil víceřádkový string. Jen se neuložil do žádné proměnné a tak hned zanikl. Narozdíl od komentářů (které se při spuštění programu přeskakují) ale Python každý víceřádkový string vytvoří, což stojí nějakou práci navíc.

Teoreticky je taky možné v rámci víceřádkového (f-)stringu spustit libovolný kód, což by v komentáři určitě jít nemělo.

In [23]:
f"""
Zde není komentář.

Zkusíme dělit nulou: {1 / 0}.
"""

ZeroDivisionError: division by zero

### Další způsob formátování stringů

Pro formátování stringů jsme si už dříve ukázali f-stringy. Ty nám umožňují do stringu dodat hodnotu libovolného Pythonovského výrazu. Ačkoliv Python umí víc způsobů, f-stringy se používají nejčastěji, protože jsou nejčitelnější a nejrychlejší.

Jejich nevýhodou ale je, že nemůžeme stejný string využít víckrát, a pokaždé do něj vložit hodnoty jiných výrazů. Právě to se nám někdy hodí při psaní modulárních SQL dotazů.

Dejme tomu, že máme tabulku studentů.

<table>
    <tr><th>Jméno</th><th>Bydliště</th><th>Studijní obor</th></tr>
    <tr><td>František Novák</td><td>Praha</td><td>Historie</td></tr>
    <tr><td>Josef Krátký</td><td>Litomyšl</td><td>Matematika</td></tr>
    <tr><td>Petr Švec</td><td>Olomouc</td><td>Alchymie</td></tr>
    <tr><td>David Kovář</td><td>Znojmo</td><td>Hudba</td></tr>
    <tr><td>Prokop Novotný</td><td>Praha</td><td>Tanec</td></tr>
    <tr><td>Marek Černý</td><td>Plzeň</td><td>Medicína</td></tr>
</table>

Podobně máme tabulku profesorů.

<table>
    <tr><th>Jméno</th><th>Bydliště</th><th>Vydaných článků</th></tr>
    <tr><td>Jiří Volf</td><td>Brno</td><td>1</td></tr>
    <tr><td>Jan Kulhavý</td><td>Praha</td><td>40</td></tr>
    <tr><td>Michal Horák</td><td>Ostrava</td><td>3</td></tr>
    <tr><td>Ondřej Veselý</td><td>Plzeň</td><td>18</td></tr>
</table>

Chceme vybrat studenty i profesory, kteří žijí v Praze. V obou případech bude dotaz podobný.

In [25]:
studenti_praha = """
SELECT jmeno
FROM studenti
WHERE bydliste = 'Praha'
"""

profesori_praha = """
SELECT jmeno
FROM profesori
WHERE bydliste = 'Praha'
"""

Protože se oba dotazy liší jen názvem tabulky, bylo by mnohem hezčí napsat si obecný dotaz pro výběr jmen s bydlištěm v Praze, a podle potřeby do něj doplnit název tabulky.

In [26]:
bydliste_praha = """
SELECT jmeno
FROM {tabulka}
WHERE bydliste = 'Praha'
"""

In [29]:
print(bydliste_praha.format(tabulka="studenti"))


SELECT jmeno
FROM studenti
WHERE bydliste = 'Praha'



In [30]:
print(bydliste_praha.format(tabulka="profesori"))


SELECT jmeno
FROM profesori
WHERE bydliste = 'Praha'



Můžeme jít ještě dál, a i místo bydliště nechat jako parametr. Jen pozor na jednoduché uvozovky okolo hodnoty tohoto parametru.

In [31]:
jmeno_podle_bydliste = """
SELECT jmeno
FROM {tabulka}
WHERE bydliste = '{bydliste}'
"""

In [33]:
print(jmeno_podle_bydliste.format(tabulka="studenti", bydliste="Brno"))


SELECT jmeno
FROM studenti
WHERE bydliste = 'Brno'



## 2. SELECT

Zatím jsme si vyráběli SQL dotazy ve stringu, ale neposílali jsme je do databáze. To přijde teď.

In [1]:
import pandas as pd
import snowflake.connector

Nejprve potřebujeme zadat jméno a heslo pro přístup do databáze. Formálně je potřeba i role, ale zde víme jak vypadá, tak ji dostaneme automaticky.

In [ ]:
name = ...  # Dopln svuj login jako string.
password = ...  # Dopln svoje heslo jako string.
role = "ROLE_" + name.upper()

Doplníme údaje o databázovém serveru a pomocí Snowflake connectoru navážeme spojení.

In [41]:
DATABASE = "COURSES"
HOST = "https://ip68917.eu-west-1.snowflakecomputing.com/"
SCHEMA = "SCH_CZECHITA"
WAREHOUSE = "COMPUTE_WH"
ACCOUNT = "ip68917.eu-west-1"

conn = snowflake.connector.connect(
    user=name,
    password=password,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA,
    role=role,
)

Vyzkoušíme jednoduchý SQL dotaz -- vybereme všechny státy z tabulky `country` ve schématu `SCH_CZECHITA`. Výsledkem je obyčejný Pandas dataframe.

In [36]:
sql = """
SELECT *
FROM SCH_CZECHITA.country
"""

pd.read_sql_query(sql, conn)

,ID,NAME
0,4,Afghanistan
1,5,Albania
2,6,Algeria
3,8,Angola
4,11,Argentina
...,...,...
132,603,United Kingdom
133,1001,Serbia
134,1002,Montenegro
135,1003,Kosovo


Protože jsme schéma uvedli už při připojení, nemusíme ho opakovat u každé tabulky.

In [42]:
sql = """
SELECT *
FROM country
"""

pd.read_sql_query(sql, conn)

,ID,NAME
0,4,Afghanistan
1,5,Albania
2,6,Algeria
3,8,Angola
4,11,Argentina
...,...,...
132,603,United Kingdom
133,1001,Serbia
134,1002,Montenegro
135,1003,Kosovo


Podobně jako při načítání souboru můžeme nějaký sloupec použít jako index. Typicky se k tomu hodí primární klíč.

In [35]:
sql = """
SELECT *
FROM country
"""

pd.read_sql_query(sql, conn, index_col="ID")

,NAME
ID,
4,Afghanistan
5,Albania
6,Algeria
8,Angola
11,Argentina
...,...
603,United Kingdom
1001,Serbia
1002,Montenegro


### Porovnání SQL a Pandas

Teď si ukážeme jak by vypadal stejný dotaz v SQL a v Pandas. Vybereme počet mrtvých (`nkill`) a zemi (`country_txt`) z tabulky `teror` ze všech útoků, kde někdo zemřel a staly se v Iráku nebo v Sýrii. Protože je jich hodně, omezíme se jen na prvních 100 podle počtu mrtvých.

In [49]:
sql = """
SELECT nkill,
       country_txt
FROM teror
WHERE nkill > 0
  AND country_txt IN ('Iraq', 'Syria')
ORDER BY nkill DESC
LIMIT 100
"""

pd.read_sql_query(sql, conn)

,NKILL,COUNTRY_TXT
0,1570,Iraq
1,953,Iraq
2,670,Iraq
3,517,Syria
4,433,Syria
...,...,...
95,49,Iraq
96,49,Iraq
97,49,Iraq
98,48,Syria


In [61]:
sql = """
SELECT *
FROM teror
"""

df = pd.read_sql_query(sql, conn)

nekdo_zemrel = df["NKILL"] > 0
zadana_zeme = df["COUNTRY_TXT"].isin(["Syria", "Iraq"])
zadane_sloupce = ["NKILL", "COUNTRY_TXT"]

df.loc[nekdo_zemrel & zadana_zeme, zadane_sloupce].sort_values(by="NKILL", ascending=False).head(100)

,NKILL,COUNTRY_TXT
8181,1570.0,Iraq
10946,953.0,Iraq
7888,670.0,Iraq
11409,517.0,Syria
44861,433.0,Syria
...,...,...
21901,49.0,Iraq
37832,49.0,Iraq
48001,49.0,Iraq
20139,48.0,Syria


Vidíme, že výsledek je téměř stejný. Jediný rozdíl je v datových typech - počet mrtvých z SQL je v celých číslech, pandy nám to daly jako floaty. Z minulé lekce můžeme tušit, že by tam mohly být nějaké chybějící hodnoty.

In [62]:
df["NKILL"].isna().sum()

3220

Je to tak.

O něco fundamentálnější rozdíl je v přístupu k datům. Pokud zadáme celý dotaz v SQL, celý výpočet běží na databázovém serveru. Pokud použijeme Pandas, běží výpočet na našem počítači. Navíc je s tím spojená práce navíc -- abychom dostali data pro Pandas, museli jsme selectnout úplně celou tabulku.

Tohle byl demonstrační příklad. V praxi bychom preferovali nechat maximum práce na databázi, aby k nám do počítače šlo co nejmenší množství dat. Pandas (případně jiné Pythonovské knihovny) pak použijeme na tento pročitěný dataset. To nám umožňí provést operace, které SQL buď přímo nepodporuje, nebo jsou v něm zbytečně těžkopádné.

## 3. JOIN

## INSERT

Je samozřejmě taky možný, ale tady se do něj pouštět nebudeme. Vyžadovalo by to přenastavit schéma aby měl každý práva zapisovat, a je trochu problém napsat to efektivně. Smyslem lekce bylo ukázat že je možné propojit Python se Snowflake (případně libovolnou jinou databází), detaily si podle potřeby každý dohledá.

## Zavření spojení

Stejně jako spojení do souboru je potřeba spojení do databáze na konci zavřít.

Opět lze použít `with` blok a nechat si ho zavřít automaticky.
```python
with snowflake.connector.connect(
    user=name,
    password=password,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA,
    role=role,
) as conn:
    pass
```
V rámci tohoto notebooku se to úplně nehodí, protože spojení potřebujeme používat na víc dotazů během celé lekce. Pokud ale víme, že do databáze posíláme jen pár dotazů které se provádějí hned po sobě, je vhodné `with` blok použít, stejně jako při práci se soubory.

In [39]:
conn.close()